### Imports

In [288]:
import gurobipy as gp
import pandas as pd

### Functions

In [289]:
def extract(data, row, fact, maximize=True):
    text = str(data.iloc[row][fact]).strip("mg%?")
    if str(text) == "nan" or len(text) == 0:
        return maximize * 1_000_000
    return float(text)

### File Input

In [290]:
df = pd.read_excel("Nutritional Facts.xlsm", index_col=0)
guide = pd.read_excel("Dietary Guidelines.xlsx", header=1, index_col=[1, 2], skiprows=[2]).drop("Unnamed: 0", axis=1)

### Restaurant / Category Selection

In [292]:
res = "Arby's"
cats = []  # ["Breakfast", "Beverages"]
real_meal = True
cat_limit = True
subset = df
if res != "":
    subset = subset[subset["Restaurant"] == res]
if cats != []:
    subset = subset[subset["Common Category"].isin(cats)]
if real_meal and not cat_limit:
    bad_cats = ["Drinks", "Beverages", "Sauces and Condiments", "Coffee", "Salad Dressing"
                "Ocean Water", "Iced Teas", "Limeades", "Soft Drinks", ]
    subset = subset[~subset["Category"].isin(bad_cats)]
subset.reset_index(inplace=True)
subset  # Contains all viable food entries

,index,Restaurant,Category,Food,Serving Size,Calories From Fat,Calories,Total Fat,Saturated Fat,Trans Fat,...,Total Fat %,Saturated Fat %,Cholesterol %,Sodium %,Total Carbohydrates %,Dietary Fiber %,Protein %,Vitamin A %,URL,Common Category
0,0,Arby's,Sandwiches,Arby's Melt,146g,110,330,12g,4.0g,.5g,...,18%,20%,12%,39%,13%,8%,36%,2%,https://fastfoodnutrition.org/arbys/arbys-melt,Sandwiches
1,1,Arby's,Sandwiches,Arby-Q Sandwich,182g,100,400,11g,3.5g,.5g,...,17%,18%,10%,52%,19%,12%,36%,4%,https://fastfoodnutrition.org/arbys/arby-q-san...,Sandwiches
2,2,Arby's,Sandwiches,Beef 'n Cheddar Classic,1 sandwich,180,450,20g,6g,1g,...,31%,30%,17%,53%,15%,8%,46%,2%,https://fastfoodnutrition.org/arbys/beef-n-che...,Sandwiches
3,3,Arby's,Sandwiches,Beer Battered Fish Sandwich,1 sandwich,210,600,23g,4g,0g,...,35%,20%,22%,76%,24%,20%,50%,NaN,https://fastfoodnutrition.org/arbys/beer-batte...,Sandwiches
4,4,Arby's,Sandwiches,Buffalo Crispy Chicken Sandwich,1 sandwich,210,500,23g,4.5g,0.0g,...,35%,23%,18%,78%,16%,16%,48%,NaN,https://fastfoodnutrition.org/arbys/buttermilk...,Sandwiches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,181,Arby's,Sides and Snacks,"Potato Cakes, 4-piece",200g,250,490,28g,4.5g,0.0g,...,43%,23%,0%,36%,15%,20%,8%,0%,https://fastfoodnutrition.org/arbys/potato-cak...,Sides
182,182,Arby's,Sides and Snacks,"Sweet Potato Waffle Fries, kids",68g,120,250,13g,2g,0g,...,20%,10%,0%,4%,10%,16%,2%,NaN,https://fastfoodnutrition.org/arbys/sweet-pota...,Sides
183,183,Arby's,Sides and Snacks,"Sweet Potato Waffle Fries, small",85g,150,310,17g,2.5g,0g,...,26%,13%,0%,5%,13%,20%,4%,NaN,https://fastfoodnutrition.org/arbys/sweet-pota...,Sides
184,184,Arby's,Sides and Snacks,"Sweet Potato Waffle Fries, medium",116g,210,430,23g,3.5g,0g,...,35%,18%,0%,8%,18%,28%,6%,NaN,https://fastfoodnutrition.org/arbys/sweet-pota...,Sides


### Requirement Selection

In [293]:
subset_clean_cols = [c.split(",")[0].strip(" %") for c in subset.columns]  # TODO: Improve by tracking indices for exact matching
guide_clean_cols = [c.split(",")[0].strip(" %") for c in guide.columns]
filters = set([c for c in subset_clean_cols if c in guide_clean_cols])

In [294]:
gender = "Male"
age = "31-50"
nutrients = dict(zip(filters, "<= >= <= <= <= >=".split(" ")))
nutrients_tmp = {"Sodium": "<="}
nutrients

{'Sodium': '>=',
 'Total Fat': '>=',
 'Dietary Fiber': '<=',
 'Protein': '<=',
 'Saturated Fat': '<=',
 'Vitamin A': '>='}

In [295]:
guide.loc[gender, age]
requirements = dict(zip(nutrients_tmp.keys(), [guide.loc[gender, age]["Sodium, mg"]]))
requirements

{'Sodium': 2300}

### Model Building

#### Model & Variables

In [296]:
m = gp.Model()
xis = [m.addVar(vtype=gp.GRB.BINARY) for _ in subset.index]  # GRB.BINARY / GRB.INTEGER

#### Constraints

In [298]:
for fact, req in requirements.items():  # Nutrition Requirements
    print(f"Constraining {fact} <= {req}")
    m.addConstr(sum((x * extract(subset, r, fact, maximize=False) for r, x in enumerate(xis))) >= req)

if real_meal:
    for i, food in enumerate(subset.iloc):  # Excludes zero calorie (from fat) entries
        m.addConstr(xis[i] <= extract(subset, i, "Calories From Fat", maximize=True))

if cat_limit:
    print()
    for cat in pd.unique(subset["Common Category"]):
        print(f"Constraining only 1 or fewer {cat.strip('s')} items.")
        m.addConstr(sum([x for i, x in enumerate(xis) if subset["Common Category"][i] == cat]) <= 1)

Constraining Sodium <= 2300

Constraining only 1 or fewer Sandwiche items.
Constraining only 1 or fewer Beverage items.
Constraining only 1 or fewer Breakfast items.
Constraining only 1 or fewer Salad items.
Constraining only 1 or fewer Condiment items.
Constraining only 1 or fewer Dessert items.
Constraining only 1 or fewer Side items.


#### Objective

In [299]:
m.ModelSense = gp.GRB.MINIMIZE
m.setObjective(sum((x * extract(subset, r, "Calories From Fat", maximize=True) for r, x in enumerate(xis))))

### Solving

In [300]:
m.optimize()
if m.status != 2:
    raise ValueError("The model is infeasible - please try again")

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 388 rows, 186 columns and 1110 nonzeros
Model fingerprint: 0xb3d75040
Variable types: 0 continuous, 186 integer (186 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Found heuristic solution: objective 190.0000000
Presolve removed 381 rows and 164 columns
Presolve time: 0.00s
Presolved: 7 rows, 22 columns, 43 nonzeros
Variable types: 0 continuous, 22 integer (22 binary)

Root relaxation: objective 1.402817e+02, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  140.28169    0    1  190.00000  140.28169  26.2%     -    0s
H    0     0                     160.0000000  140.28169  12.3%     -    0s

Cutting planes:
  Cover: 1

Explored 1 nodes (2 simplex ite

### Optimal Objective

In [301]:
print(m.ObjVal, "calories from fat")
m.printAttr('X')
best_choices = [i for i, x in enumerate(xis) if x.x > 0]

160.0 calories from fat

    Variable            X 
-------------------------
          C5            1 
         C54            1 
        C137            1 


### Corresponding Meal(s)

In [302]:
print(subset.iloc[best_choices]["Food"], ":")
selection = subset.iloc[best_choices]
selection = selection[selection["Protein"] > '0g']
selection = selection[selection["Dietary Fiber"] > '0g']
selection

5      Buffalo Roast Chicken Sandwich
54                     Chocolate Milk
137            Light Italian Dressing
Name: Food, dtype: object :


,index,Restaurant,Category,Food,Serving Size,Calories From Fat,Calories,Total Fat,Saturated Fat,Trans Fat,...,Total Fat %,Saturated Fat %,Cholesterol %,Sodium %,Total Carbohydrates %,Dietary Fiber %,Protein %,Vitamin A %,URL,Common Category
5,5,Arby's,Sandwiches,Buffalo Roast Chicken Sandwich,1 sandwich,130,360,14g,3.5g,0g,...,22%,18%,22%,62%,12%,12%,48%,NaN,https://fastfoodnutrition.org/arbys/buffalo-ro...,Sandwiches
54,54,Arby's,Beverages,Chocolate Milk,218g,20,150,3g,2.0g,0.0g,...,5%,10%,3%,7%,9%,4%,14%,8%,https://fastfoodnutrition.org/arbys/shamrock-f...,Beverages
